In [10]:
from __future__ import print_function
import keras
from keras.applications import ResNet50
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
import os

# Установка параметров нейросети
batch_size = 64
num_classes = 10
epochs = 2

# Загрузка данных ImageNet
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Создание объекта ImageDataGenerator
datagen = ImageDataGenerator(rescale=1.0/255.0,
                             rotation_range=10,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=False,
                             vertical_flip=False)

# Создание генераторов для обучающей и тестовой выборок
train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)
test_generator = datagen.flow(x_test, y_test, batch_size=batch_size)

# Создание предварительно обученной модели ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Создание модели для Featurization
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))

# Компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Обучение модели
model.fit_generator(train_generator,
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=len(x_test) // batch_size)

# Оценка производительности модели на тестовых данных
score = model.evaluate_generator(test_generator, steps=len(x_test) // batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/2


<ipython-input-10-5ccbb8cfd211>:48: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


781/781 [==============================] - 112s 99ms/step - loss: 144.0063 - accuracy: 0.0219 - val_loss: 153.8680 - val_accuracy: 0.0057
Epoch 2/2
781/781 [==============================] - 66s 85ms/step - loss: 125.7398 - accuracy: 0.0000e+00 - val_loss: 131.1112 - val_accuracy: 0.0026


<ipython-input-10-5ccbb8cfd211>:55: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(test_generator, steps=len(x_test) // batch_size)


Test loss: 131.27549743652344
Test accuracy: 0.0023036859929561615


**Выводы:**
- Не совсем понял отражаемую статистику при использовании оптимизатора **Adam** (loss: 149955.2500 - accuracy: 0.0000e+00 - val_loss: 185992.7500 - val_accuracy: 0.0000e+00), точность оказалась очень мала на двух эпохах.
- В связи с тем, что **Adam** не справлялся с задачей, я заменил его на **SGD**, оказалось, что для достижения результата в точности 0.87+ достаточно одной эпохи с батчами равными 64 на этом оптимизаторе. Почему такая разница?
- Еще мне оказалось непонятно, почему указано в количестве классов в уроке в последней нейронке 1000 (num_classes = 1000), если в **cifar10** их соответственно 10?
- Кстати интересно, когда поменял на корректное количество классов, оптимизатор **Adam** заработал лучше, но точность у него оказалась все равно гораздо ниже чем у **SGD**.

**2 задание:**
- Если говорить о **Mnist**, то надо было бы поменять количество каналов, и в целом можно было бы поиграться с дата-генератором, насчет входа не совсем понял, mnist содержит фото 28 на 28, но Resnet принимает на вход 224 на 224, в таком случае вход остается (224, 224, 1)?
- Если говорить о **Cifar100**, то пришлось бы в целом поменять только количество классов с 10 на 100.
- А вот насчет **IMAGENET** я не знаю, потому что этот датасет насколько я понял является мультиклассовым (картинки могут содержать разные объекты), в данном случае как надо изменить параметры, т.е. что делать с количеством классов например?